# Projeto 2 - NLP

-----

Nome:  
Turma:

Os segundo projeto do módulo de Dados não estruturados II será focado no processamento de linguagem natural! Usaremos os algoritmos aprendidos e as técnicas vistas na segunda parte do curso para extrairmos informações relevantes de texto. Mais precisamente, de publicações no Twitter.

## Os Dados

Utilizaremos um Dataset obtido do Twitter com 100K postagens entre os dias 01/08/2018 e 20/10/2018. Cada postagem é classificada como **positiva**, **negativa** ou **neutra**.  

Dois arquivos serão disponilizados para o desenvolvimento dos modelos, um para treino/validação e outro para submissão. Os arquivos se encontram na pasta */Dados/train* e */Dados/subm*, respectivamente.

Descrição das colunas:

- **id**: ID único para o tweet  
- **tweet_text**: Texto da publicação no Twitter  
- **tweet_date**: Data da publicação no Twitter  
- **sentiment**: 0, se negativo; 1, se positivo; 2, se neutro  
- **query_used**: Filtro utilizado para buscar a publicação

## O Problema

Você deverá desenvolver um modelo para detectar o sentimento de uma publicação do Twitter a classificando em uma das três categorias: **positiva**, **negativa** ou **neutra**. O texto da publicação está disponível na coluna "tweet_text". Teste pelo menos 2 técnicas de NLP diferentes e escolha a métrica de avaliação que julgar mais pertinente.  

Escolha o melhor modelo e gere uma base a partir dos dados de submissão, que estão no caminho ```Dados/subm/Subm3Classes.csv```, com o seguinte formato:


|id|sentiment_predict
|-|-|
|12123232|0
|323212|1
|342235|2

Salve essa tabela como um arquivo csv com o nome ```<nome>_<sobrenome>_nlp_degree.csv``` e submeta-o como parte da entrega final do projeto.  

Para ajudar no desenvolvimento, é possível dividir o projeto em algumas fases:

- **Análise de consistência dos dados**: analise se os dados estão fazendo sentido, se os campos estão completos e se há dados duplicados ou faltantes. Se julgar necessário, trate-os.    


- **Análise exploratória**: analise a sua base como um todo, verifique o balanceamento entre as classes e foque, principalmente, na coluna ```tweet_text```.    


- **Pré-processamento e transformações**: projetos de NLP exigem um considerável pré-processamento. Foque no tratamento da string do texto. Procure começar com tratamentos simples e adicione complexidade gradualmente. Nessa etapa você testará diferentes técnicas de transformações, como o Bag Of Words e o TF-IDF.    


- **Treinamento do modelo**: depois das transformações, você poderá executar o treinamento do modelo classificador. Nessa etapa o problema se torna semelhante aos abordados nos módulos anteriores. Você pode testar diversos classificadores como RandomForest, AdaBoost, entre outros. Otimize os hiperparâmetros do modelo com técnicas como a GridSearch e a RandomizedSearch.    


- **Conclusões**: descreva, em texto, as conclusões sobre os seus estudos. O modelo é capaz de identificar o sentimento das publicações? É possível extrapolar o modelo para outros contextos, como a análise de sentimento de uma frase qualquer? Pense em questões pertinentes e relevantes que você tenha obtido durante o desenvolvimento do projeto!     



## Critérios de avaliação

Os seguintes itens serão avaliados:

1. Desenvolvimento das etapas descritas acima;


2. Reprodutibilidade do código: seu código será executado e precisa gerar os mesmos resultados apresentados por você;


3. Clareza: seu código precisa ser claro e deve existir uma linha de raciocínio direta. Comente o código em pontos que julgar necessário para o entendimento total;


4. Justificativa das conclusões obitdas: não existirá certo ou errado, mas as decisões e as conclusões precisam ser bem justificadas com base nos resultados obtidos.  

O desempenho do modelo **não** será considerado como critério de avaliação.  

## Informações gerais

- O projeto deve ser desenvolvido individualmente ou em grupo;



- Entrega através do Class: Processamento Digital de Imagens - Definições e Fundamentos
 -> Exercícios -> Projeto 2


Anexar, na entrega, o notebook de desenvolvimento e o arquivo .csv de submissão, da seguinte forma:  

Criar um arquivo zip com:
- notebook: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2.ipynb```   
- csv: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2_submissao.csv```


## Dicas

### Base de treino e submissão

A base de submissão não possui a variável de saída, portanto ela será utilizada **apenas** para gerar o arquivo que acompanha a submissão do projeto.      

### Tente encontrar possíveis vieses

É muito comum que modelos de NLP possuam fortes vieses, como a tendência de relacionar palavras específicas com alguma classe de saída. Tente encontrar vieses no seu estudo, isso pode ajudar a tirar boas conclusões. o campo "query_used" pode ser útil para essa análise.  

### O pré-processamento é a chave para um bom desempenho

Essa é a etapa que mais vai contribuir para o desempenho do seu modelo. Seja criativo e desenvolva essa etapa de uma maneira que seja fácil de aplicar o mesmo processamento para uma nova base, você terá que fazer isso para gerar a base de submissão.

-------

# Bibliotecas

In [6]:
from IPython.display import clear_output

In [7]:
!pip install nltk
!pip install scikit-learn
!pip install keras
!pip install tensorflow

clear_output(True)
print("Success!")

Success!


In [9]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import re
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

nltk.download("stopwords")
nltk.download('punkt')
pio.renderers
pio.renderers.default = "notebook_connected"

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Análise de consistência dos dados

- Não foram encontrados dados nulos;
- Foram encontrados dados duplicados porque esses dados são resultados de diferentes queries;
- As queries possuem alto víes. Cada query possui apenas um sentimento relacionado.

## Leitura dos dados

In [ ]:
training_file_path = "https://raw.githubusercontent.com/lucas54neves/twitter-sentiment-analysis/main/src/Dados/train/Train3Classes.csv"

df = pd.read_csv(training_file_path)

df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:)


In [ ]:
df.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used'], dtype='object')

## Pré-processamento para análise de consistência dos dados e para análise exploratória

In [ ]:
_df = df.copy()

def update_data(row, total):
    row["tweet_text_length"] = len(row["tweet_text"])
    row["tweet_time"] = re.search("\d{2}:\d{2}:\d{2}", row["tweet_date"]).group(0)
    
    hour = int(row["tweet_time"][:2])
    if hour < 6:
        row["tweet_period"] = "Dawn"
    elif hour < 12:
        row["tweet_period"] = "Morning"
    elif hour < 18:
        row["tweet_period"] = "Afternoon"
    else:
        row["tweet_period"] = "Night"

    if row["sentiment"] == 0:
        row["sentiment_name"] = "Negative"
    elif row["sentiment"] == 1:
        row["sentiment_name"] = "Positive"
    else:
        row["sentiment_name"] = "Neutral"

    if row.name % 500 == 0:
        value = (row.name + 1) / total * 100
        print(f"Progress: {value:.2f}%")
        clear_output(True)
    
    return row

total = len(_df)

_df = _df.apply(
    lambda row: update_data(row, total),
    axis=1
)

clear_output(True)
print("Progress: 100.00%")

Progress: 100.00%


## Verifica se tem valores nulos

Não foram encontrados dados nulos.

In [ ]:
_df.isnull().sum()

id                   0
tweet_text           0
tweet_date           0
sentiment            0
query_used           0
tweet_text_length    0
tweet_time           0
tweet_period         0
sentiment_name       0
dtype: int64

## Verifica valores duplicados

Dados duplicados foram encontrados porque foram buscados por queries diferentes.

In [ ]:
for column in _df.columns:
    exists_duplicated_values = _df[column].duplicated().sum()
    print(f"{column}: {exists_duplicated_values}")

id: 13
tweet_text: 816
tweet_date: 11713
sentiment: 94997
query_used: 94986
tweet_text_length: 94638
tweet_time: 44804
tweet_period: 94996
sentiment_name: 94997


In [ ]:
_df["id"].duplicated().sum()

13

In [ ]:
duplicated_ids = _df[_df["id"].duplicated()]["id"]

_df.query("id in @duplicated_ids").sort_values("id")

,id,tweet_text,tweet_date,sentiment,query_used,tweet_text_length,tweet_time,tweet_period,sentiment_name
39184,1037082879438729216,O que é #FATO ou #FAKE na entrevista de Gerald...,Tue Sep 04 21:00:01 +0000 2018,2,#fato,118,21:00:01,Night,Neutral
38913,1037082879438729216,O que é #FATO ou #FAKE na entrevista de Gerald...,Tue Sep 04 21:00:01 +0000 2018,2,jornaloglobo,118,21:00:01,Night,Neutral
35012,1037837855433928704,Veja o que é #FATO ou #FAKE na entrevista de E...,Thu Sep 06 23:00:01 +0000 2018,2,#fato,114,23:00:01,Night,Neutral
20226,1037837855433928704,Veja o que é #FATO ou #FAKE na entrevista de E...,Thu Sep 06 23:00:01 +0000 2018,2,jornaloglobo,114,23:00:01,Night,Neutral
39324,1038222887432273920,Veja o que é #FATO ou #FAKE na entrevista de F...,Sat Sep 08 00:30:00 +0000 2018,2,jornaloglobo,121,00:30:00,Dawn,Neutral
83565,1038222887432273920,Veja o que é #FATO ou #FAKE na entrevista de F...,Sat Sep 08 00:30:00 +0000 2018,2,#fato,121,00:30:00,Dawn,Neutral
69100,1038494427503837185,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 18:29:00 +0000 2018,2,#fato,121,18:29:00,Night,Neutral
35849,1038494427503837185,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 18:29:00 +0000 2018,2,jornaloglobo,121,18:29:00,Night,Neutral
62236,1038570177191993344,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 23:30:00 +0000 2018,2,#fato,120,23:30:00,Night,Neutral
93665,1038570177191993344,Veja o que é #FATO ou #FAKE na entrevista de H...,Sat Sep 08 23:30:00 +0000 2018,2,jornaloglobo,120,23:30:00,Night,Neutral


## Análise dos dados pela query

As queries possuem alto víes. Cada query possui apenas um sentimento relacionado.

In [ ]:
queries = _df["query_used"].unique()

queries

array(['folha', ':(', 'exame', ':)', '#fato', 'g1', '#novidade',
       '#noticia', 'estadao', 'jornaloglobo', '#curiosidade',
       '#oportunidade', 'veja', '#trabalho'], dtype=object)

In [ ]:
sentiment_by_queries = []

for query in queries:
    df_by_query = _df[_df["query_used"] == query]

    print("=" * 40)
    print(f"Query: {query}")

    sentiment_counted = df_by_query['sentiment_name'].value_counts()
    sentiment_counted_dict = sentiment_counted.to_dict()

    for (key, value) in list(sentiment_counted_dict.items()):
        print(f"Sentiment: {key} | Counted: {value}")

        sentiment_by_query = {
            "sentiment_name": key,
            "counted": value,
            "query": query,
        }

        sentiment_by_queries.append(sentiment_by_query)

Query: folha
Sentiment: Neutral | Counted: 5004
Query: :(
Sentiment: Negative | Counted: 31696
Query: exame
Sentiment: Neutral | Counted: 3417
Query: :)
Sentiment: Positive | Counted: 31678
Query: #fato
Sentiment: Neutral | Counted: 3471
Query: g1
Sentiment: Neutral | Counted: 3439
Query: #novidade
Sentiment: Neutral | Counted: 920
Query: #noticia
Sentiment: Neutral | Counted: 1114
Query: estadao
Sentiment: Neutral | Counted: 3880
Query: jornaloglobo
Sentiment: Neutral | Counted: 2374
Query: #curiosidade
Sentiment: Neutral | Counted: 381
Query: #oportunidade
Sentiment: Neutral | Counted: 2455
Query: veja
Sentiment: Neutral | Counted: 2141
Query: #trabalho
Sentiment: Neutral | Counted: 3030


In [ ]:
sentiment_by_queries_df = pd.DataFrame(sentiment_by_queries)

In [ ]:
data = sentiment_by_queries_df.copy()

fig = px.bar(
    data,
    x="query",
    y="counted",
    color="sentiment_name",
    title='Query by number of tweets',
    barmode='group',
)

fig.update_xaxes(title="Query")
fig.update_yaxes(title="Number of tweets")

fig.show()

# Análise exploratória

## Verifica tamanho do campo ```tweet_text```

Verificando o tamanho do campo ```tweet_text``` observa-se que tweets com maior tamanho em média são neutros, os tweets com menor tamanho em média são negativos e os tweets positivos ficaram com tamanho intermediário.

In [ ]:
data = _df.groupby(by="sentiment_name").mean().reset_index()[["sentiment_name", "tweet_text_length"]]

fig = px.line(data, x="sentiment_name", y="tweet_text_length", title='Sentiment by tweet text length')

fig.update_xaxes(title="Sentiment")
fig.update_yaxes(title="Tweet text length")

fig.show()

## Verificando a hora do tweet

Verificando o período de cada tweet, foi possível verificar a maioria dos tweet no período do madrugada são negativos. Nos outros períodos, os sentimentos se distribuem de forma mais homogênea.

In [ ]:
data = _df[["sentiment_name", "tweet_period"]]
data = data.groupby(by=["sentiment_name", "tweet_period"]).size().reset_index()
data.columns = ["sentiment_name", "tweet_period", "total"]

fig = px.bar(
    data,
    x="sentiment_name",
    y="total",
    color="tweet_period",
    title='Sentiment by tweet period',
    barmode='group',
)

fig.update_xaxes(title="Sentiment")
fig.update_yaxes(title="Tweet period")

fig.show()

## Verificar balanceamento

In [ ]:
_df.groupby("sentiment")["id"].count()

sentiment
0    31696
1    31678
2    31626
Name: id, dtype: int64

In [ ]:
_df.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used',
       'tweet_text_length', 'tweet_time', 'tweet_period', 'sentiment_name'],
      dtype='object')

# Pré-processamento e transformações

In [ ]:
stops = stopwords.words("portuguese")

In [ ]:
def update_data(row, total):
    # Converte todas as letras em minúsculas
    text_in_lower = row["tweet_text"].lower()
    tokens = word_tokenize(text_in_lower, language="portuguese")
    words_without_stopwords = [word for word in tokens if word not in stops]
    words_without_punctuation = [word for word in words_without_stopwords if word not in string.punctuation]

    row["tweet_text_formatted"] = " ".join(words_without_punctuation)

    if row.name % 500 == 0:
        value = (row.name + 1) / total * 100
        print(f"Progress: {value:.2f}%")
        clear_output(True)
    
    return row

total = len(_df)

_df = _df.apply(
    lambda row: update_data(row, total),
    axis=1
)

clear_output(True)
print("Progress: 100.00%")

Progress: 40.53%


## Representação textual

In [ ]:
_df.columns

In [ ]:
token = Tokenizer(num_words=100)
token.fit_on_texts(_df["tweet_text_formatted"])

In [ ]:
X = token.texts_to_sequences(_df["tweet_text_formatted"])
X = pad_sequences(X, padding="post", maxlen=100)

In [ ]:
X

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(_df["sentiment"])

In [ ]:
print(y)

In [ ]:
y = np_utils.to_categorical(y)

In [ ]:
print(y)

# Treinamento do modelo

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3
)

In [ ]:
model = Sequential()

model.add(
    Embedding(
        input_dim=len(token.word_index),
        output_dim=128,
        input_length=X.shape[1],
    )
)

model.add(
    SpatialDropout1D(0.2)
)

model.add(
    LSTM(
        units=196,
         dropout=0.2,
         recurrent_dropout=0,
         activation="tanh",
         recurrent_activation="sigmoid",
         unroll=False,
         use_bias=True,
    )
)

model.add(
    Dense(
        units=3,
        activation="softmax",
    )
)

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
    ]
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=30,
    verbose=True,
    validation_data=(
        X_test,
        y_test,
    )
)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
prev = model.predict(X_test)

print(prev)

# Conclusões